In [1]:
import geemap
import ee
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [21]:
roi = Map.draw_last_feature.geometry()

In [4]:
rectangle = ee.Geometry.Rectangle([73, 3, 136, 54])  # ee.Geometry.Rectangle(minLng, minLat, maxLng, maxLat)
roi = ee.Geometry(rectangle, None, False)
Map.addLayer(roi, {}, 'rectangle')

In [22]:
visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 3000
}

In [28]:
image = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
            .filterDate('2018-05', '2018-08').filterBounds(roi).sort('CLOUD_COVER', True)
Map.addLayer(image.first(), visParams, 'img')

In [26]:
image.aggregate_array('system:id').getInfo()

['LANDSAT/LC08/C01/T1_SR/LC08_121040_20130328',
 'LANDSAT/LC08/C01/T1_SR/LC08_121040_20130428',
 'LANDSAT/LC08/C01/T1_SR/LC08_121040_20130514',
 'LANDSAT/LC08/C01/T1_SR/LC08_121040_20130615',
 'LANDSAT/LC08/C01/T1_SR/LC08_121040_20130701',
 'LANDSAT/LC08/C01/T1_SR/LC08_121040_20130717',
 'LANDSAT/LC08/C01/T1_SR/LC08_121040_20130802',
 'LANDSAT/LC08/C01/T1_SR/LC08_121040_20130818',
 'LANDSAT/LC08/C01/T1_SR/LC08_121040_20130919',
 'LANDSAT/LC08/C01/T1_SR/LC08_121040_20131005',
 'LANDSAT/LC08/C01/T1_SR/LC08_121040_20131021',
 'LANDSAT/LC08/C01/T1_SR/LC08_121040_20131106',
 'LANDSAT/LC08/C01/T1_SR/LC08_121040_20131122',
 'LANDSAT/LC08/C01/T1_SR/LC08_121040_20131208',
 'LANDSAT/LC08/C01/T1_SR/LC08_121040_20131224',
 'LANDSAT/LC08/C01/T1_SR/LC08_121040_20140109',
 'LANDSAT/LC08/C01/T1_SR/LC08_121040_20140125',
 'LANDSAT/LC08/C01/T1_SR/LC08_121040_20140210',
 'LANDSAT/LC08/C01/T1_SR/LC08_121040_20140314',
 'LANDSAT/LC08/C01/T1_SR/LC08_121040_20140415',
 'LANDSAT/LC08/C01/T1_SR/LC08_121040_201

In [ ]:
Map.addLayer(temp_mean.subtract(273).clip(roi), {}, "a")

In [ ]:
clip_dow_merge(roi, images2, r'E:/Data/temp/mean1', 500)

分割成99份, 开始下载:
Generating URL ...
Please wait ...
Data downloaded to E:\Data\temp\mean1_mk\temp_0.tif
Generating URL ...
Please wait ...
Data downloaded to E:\Data\temp\mean1_mk\temp_1.tif
Generating URL ...
Please wait ...
Data downloaded to E:\Data\temp\mean1_mk\temp_2.tif
Generating URL ...
Please wait ...
Data downloaded to E:\Data\temp\mean1_mk\temp_3.tif
Generating URL ...
Please wait ...
Data downloaded to E:\Data\temp\mean1_mk\temp_4.tif
Generating URL ...
Please wait ...
Data downloaded to E:\Data\temp\mean1_mk\temp_5.tif
Generating URL ...
Please wait ...
Data downloaded to E:\Data\temp\mean1_mk\temp_6.tif
Generating URL ...
Please wait ...
Data downloaded to E:\Data\temp\mean1_mk\temp_7.tif
Generating URL ...
Please wait ...
Data downloaded to E:\Data\temp\mean1_mk\temp_8.tif
Generating URL ...
Please wait ...
Data downloaded to E:\Data\temp\mean1_mk\temp_9.tif
Generating URL ...
An error occurred while downloading.
Object of type int32 is not JSON serializable
Generating URL

In [11]:
def clip_dow_merge(geo: ee.Geometry, image: ee.Image, outfile: str, scale: int,
                   crs='epsg:4326', sep=0.25):
    """

    Args:
        geo: ee.Geometry, 需要下载的区域矢量几何
        image: ee.Image, 需要下载的影像
        outfile: str, 输出文件路径和名称，不需要文件后缀，下载的影响默认后缀为tif
        scale: int, 下载时的像元大小
        crs: str, 下载影像的投影，默认为 'epsg:4326' wgs1984投影
        sep: float, 单波段10m分辨率像元的影像裁剪大小(单位：经纬度)，默认为0.25
    Returns: None

    """
    import os
    import numpy as np
    import rasterio
    from glob import glob
    from rasterio.merge import merge
    import shutil
    import geemap
    import math
    bounds = geo.bounds()
    bands = image.bandNames().size().getInfo()
    poy = np.array(bounds.coordinates().getInfo()[0])
    min_x = poy[:, 0].min()
    max_x = poy[:, 0].max()
    min_y = poy[:, 1].min()
    max_y = poy[:, 1].max()
    step = scale / 10 * sep / (int(math.sqrt(bands))+1)
    end_x = int((max_x - min_x) / step) + 1
    end_y = int((max_y - min_y) / step) + 1
    polys = []
    for i in range(end_y):
        y1 = min_y + step * i
        y2 = min_y + step * (i + 1)
        if y2 > max_y:
            y2 = max_y
        for j in range(end_x):
            x1 = min_x + step * j
            x2 = min_x + step * (j + 1)
            if x2 > max_x:
                x2 = max_x
            poly = ee.Geometry(ee.Geometry.Rectangle([x1, y1, x2, y2]), None, False)
            polys.append(poly)
    if len(polys) > 1:
        print(f"分割成{len(polys)}份, 开始下载:")
        path = outfile+'_mk'
        if not os.path.exists(path):
            os.makedirs(path)
        for j, i in enumerate(polys):
            geemap.ee_export_image(image, path+f'/temp_{j}.tif', scale=scale, crs=crs, region=i)
        files = glob(path+"/*.tif")
        src_files_to_mosaic = []
        for tif_f in files:
            src = rasterio.open(tif_f)
            src_files_to_mosaic.append(src)
        mosaic, out_trans = merge(src_files_to_mosaic)
        out_meta = src.meta.copy()
        out_meta.update({"driver": "GTiff",
                         "height": mosaic.shape[1],
                         "width": mosaic.shape[2],
                         "transform": out_trans,
                         })
        with rasterio.open(outfile+".tif", "w", **out_meta) as dest:
            dest.write(mosaic)
        for src in src_files_to_mosaic:
            src.close()
        shutil.rmtree(path)
    else:
        geemap.ee_export_image(image, outfile+'.tif', scale=scale, crs=crs, region=geo)
    print("download successful !!!")
